tmp

In [1]:
import pandas as pd
import scipy.io.wavfile as wav
import numpy as np
from os import path

In [56]:
folder = 'wygibasy'
bpm = 180

In [52]:
piosenka = create_song(bpm, folder)

/usr/lib/python3/dist-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [53]:
piosenka

array([[ 3,  2],
       [ 0, -5],
       [ 2,  7],
       ..., 
       [ 0,  0],
       [ 0,  0],
       [ 0,  0]], dtype=int16)

In [54]:
wav.write('wygibasy/piosenka2.wav', 44100, piosenka)

In [55]:
!!aplay 'wygibasy/piosenka2.wav'

["Playing WAVE 'wygibasy/piosenka2.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]

In [66]:
defs = pd.read_json('wygibasy/defs.txt', typ = 'series')

In [67]:
defs

bpm    120
dtype: int64

In [68]:
defs['bpm']

120

In [69]:
defs['dupa']

KeyError: 'dupa'

In [64]:
type(defs)

pandas.core.series.Series

In [3]:
def concatenate_one_type_of_sample(column_of_track, length_of_column, folder, next_sample):
    column = np.zeros((length_of_column, 2))
    # check the number of sample
    num_of_sample = list(set(column_of_track).difference(['--']))[0]
    sample = wav.read(path.join(folder, 'sample' + num_of_sample + '.wav'))
    # number of bits in the sample
    n = sample[1].shape[0]
    # if length of sample is bigger than number of sounds (?) per bit we add more columns at the end of array
    if n > next_sample:
        how_many_to_add = n - next_sample
        column = np.zeros((length_of_column + how_many_to_add, 2))
    # concatenate sample and 'empty sounds' in order
    for i, j in enumerate(column_of_track):
        if j != '--':
            column[i * next_sample : i * next_sample + n, :] += sample[1]
    column = np.array(column, dtype = 'int16')
    return column

In [4]:
def concatenate_all_samples_in_track(track, bpm, folder):
    nrow_track = track.shape[0]
    ncol_track = track.shape[1]
    # bit per second
    bps = bpm / 60
    # all samples have the same sample rate
    f = 44100
    next_sample = int(f // bps)
    N = next_sample * nrow_track
    list_of_samples = list()
    for k in range(ncol_track):
        column_of_track = track.loc[:, k]
        column = concatenate_one_type_of_sample(column_of_track, N, folder, next_sample)
        list_of_samples.append(column)
    return [N, list_of_samples]

In [5]:
def merge_all_concatenated_samples(list_of_samples):
    k = len(list_of_samples)
    # look for maximum number of rows in all samples
    num_of_rows = [None] * k
    for i in range(k):
        num_of_rows[i] = list_of_samples[i].shape[0]
    max_of_rows = max(num_of_rows)                 
    # append all arrays with zeros to have the same shape
    for i in range(k):             
        n = list_of_samples[i].shape[0]
        if n != max_of_rows:
            list_of_samples[i] = np.vstack((list_of_samples[i], np.zeros((max_of_rows - n, 2))))  
    #mix all sample into one array (average)
    sample_all = np.zeros((max_of_rows, 2))
    for i in range(k):
        sample_all += 1/k * list_of_samples[i]
    sample_all = np.array(sample_all, dtype = 'int16')
    return sample_all

In [6]:
def create_track(track_name, bmp, folder):
    path_to_track = path.join(folder, track_name)
    track = pd.read_csv(path_to_track, sep = ' ', header = None, dtype = str)
    raw_num_of_rows_and_list_of_samples = concatenate_all_samples_in_track(track, bpm, folder)
    raw_num_of_rows = raw_num_of_rows_and_list_of_samples[0]
    track_array = merge_all_concatenated_samples(raw_num_of_rows_and_list_of_samples[1])
    true_num_of_rows = track_array.shape[0]
    return [track_array, raw_num_of_rows, true_num_of_rows]

In [7]:
def take_track_names(folder):
    dir_to_song = path.join(folder, "song.txt")
    song = pd.read_csv(dir_to_song, header = None, dtype = str)
    # all unique tracks used in the song
    track_numbers = list(set(song.loc[:, 0]))
    num_of_tracks = len(track_numbers)
    name_of_track = [None] * num_of_tracks
    for i in range(num_of_tracks):
        name_of_track[i] = 'track' + track_numbers[i] + '.txt'
    return name_of_track

In [8]:
def create_all_track(bpm, folder):
    name_of_track = take_track_names(folder)
    # unique tracks
    num_of_tracks = len(name_of_track)
    dict_of_tracks = {}
    for i in range(num_of_tracks):
        dict_of_tracks[name_of_track[i]] = create_track(name_of_track[i], bpm, folder)
    return dict_of_tracks

In [36]:
def create_song(bpm, folder):
    dict_of_tracks = create_all_track(bpm, folder)
    dir_to_song = path.join(folder, "song.txt")
    song = pd.read_csv(dir_to_song, header = None, dtype = str)
    # all tracks used in the song in the order
    track_names = 'track' + song.loc[:, 0] + '.txt'
    # the length of the song = sum of raw length of every track + eventually additional ending of the last track
    length_of_song = 0
    for i, j in enumerate(track_names):
        if i != len(track_names) - 1:
            length_of_song += dict_of_tracks[j][1]
        else:
            length_of_song += dict_of_tracks[j][2]
    song = np.zeros((length_of_song, 2))
    for i, j in enumerate(track_names):
        if i == 0:
            where_to_start = 0
            raw_length_of_track = dict_of_tracks[j][1]
            true_length_of_track = dict_of_tracks[j][2]
            song[where_to_start:true_length_of_track, :] += 0.5 * dict_of_tracks[j][0]
            where_to_start += raw_length_of_track 
        else:
            raw_length_of_track_new = dict_of_tracks[j][1]
            true_length_of_track_new = dict_of_tracks[j][2]
            song[where_to_start:(where_to_start + true_length_of_track_new), :] += 0.5 * dict_of_tracks[j][0]
            where_to_start += raw_length_of_track_new
    song = np.array(song, dtype = 'int16')
    return song

In [76]:
def read_defs(folder):
    defs = pd.read_json(path.join(folder, 'defs.txt'), typ = 'series')
    return defs
    
defs = read_defs(folder)

bpm = defs['bpm']

In [77]:
bpm

150

In [15]:
import numpy as np
f = 440 # Hz
T = 1 # s
fs = 44100 # Hz, sampling frequency
S = pow(2, 1/12)
t = np.linspace(0, T, T*fs)
y_A4 = np.sin(2*np.pi*f*t)
y_C5 = np.sin(2*np.pi*f*(S**3)*t)
y_E5 = np.sin(2*np.pi*f*(S**7)*t)
# y = np.r_[y_A4, y_C5, y_E5]
y = y_A4+y_C5+y_E5

import scipy.io.wavfile
scipy.io.wavfile.write('/var/tmp/test.wav',
                       fs,
                       np.int16(y/max(np.abs(y))*32767))
!!aplay /var/tmp/test.wav

["Playing WAVE '/var/tmp/test.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Mono"]

In [16]:
len(t)

44100

In [11]:
def create_frequency(note_string):
    letter = note_string[0]
    sharp = note_string[1]
    number = int(note_string[2])
    f_basic = 440
    S = 2 ** (1 / 12)
    # dictionary with powers
    dict_of_sound = {'A': 0, 'B': 2, 'C' : 3, 'D' : 5, 'E' : 7, 'F' : 8, 'G' : 10}
    f = f_basic * (S ** dict_of_sound[letter])
    if sharp == '#':
        f *= S
    diff_number = number - 4
    f *= 2 ** diff_number
    return f 

In [14]:
create_frequency('B-5')

987.7666025122483

In [23]:
np.array(y)

array([ 0.        ,  0.23092505,  0.46036327, ...,  1.85278018,
        1.93316752,  1.99945861])

In [24]:
y.vstack(y)

AttributeError: 'numpy.ndarray' object has no attribute 'vstack'

In [28]:
np.transpose(np.vstack((y, y)))

array([[ 0.        ,  0.        ],
       [ 0.23092505,  0.23092505],
       [ 0.46036327,  0.46036327],
       ..., 
       [ 1.85278018,  1.85278018],
       [ 1.93316752,  1.93316752],
       [ 1.99945861,  1.99945861]])

In [34]:
f = create_frequency('A-4')
#do 1?
t = np.linspace(0, 1, 22050)
y = np.sin(2 * np.pi * f * t)
y = np.int16(y / max(np.abs(y)) * 32767)
sample = np.transpose(np.vstack((y, y)))

In [35]:
sample

array([[    0,     0],
       [ 4097,  4097],
       [ 8131,  8131],
       ..., 
       [-8131, -8131],
       [-4097, -4097],
       [    0,     0]], dtype=int16)

In [39]:
scipy.io.wavfile.write('/var/tmp/test.wav',
                       44100,
                       sample)
!!aplay /var/tmp/test.wav

["Playing WAVE '/var/tmp/test.wav' : Signed 16 bit Little Endian, Rate 44100 Hz, Stereo"]